# 6.10.1 Ecuaciones diferenciales de compatibilidad tridimensionales

|Quién | Correo | Fecha | Qué hizo |
| ---  | ---   | --- | ---      |
|Diego Andrés Alvarez Marín | <daalvarez@unal.edu.co>  | Marzo 26, 2023 | Código inicial |

## Ecuaciones de compatibilidad de Saint-Venant

In [1]:
from sympy import sin, cos, diff, Function, simplify, symbols, Matrix, expand

Se definen las variables y funciones de interés:

In [2]:
r, t, z = symbols("r, theta, z")

er  = Function('varepsilon_r')(r, t, z)
et  = Function('varepsilon_θ')(r, t, z)
ez  = Function('varepsilon_z')(r, t, z)
grt = Function('gamma_rθ')(r, t, z);        ert = grt/2
grz = Function('gamma_rz')(r, t, z);        erz = grz/2
gtz = Function('gamma_θz')(r, t, z);        etz = gtz/2

Se define la matriz de transformación $\boldsymbol{T}$:

In [3]:
T = Matrix([[ cos(t), -sin(t),       0 ],     
            [ sin(t),  cos(t),       0 ],      
            [      0,       0,       1 ]])

Se define el tensor de deformaciones en coordenadas cilíndricas $\underline{\underline{\boldsymbol{\varepsilon}_{\text{cil}}}}$:

In [4]:
eps_cil = Matrix([[  er, ert, erz ],
                  [ ert,  et, etz ],                       
                  [ erz, etz,  ez ]])

Se convierte el tensor de deformaciones $\underline{\underline{\boldsymbol{\varepsilon}_{\text{cil}}}}$ de la base $\{\hat{\boldsymbol{r}}, \hat{\boldsymbol{\theta}}, \hat{\boldsymbol{z}}\}$ a la base $\{\hat{\boldsymbol{i}}, \hat{\boldsymbol{j}}, \hat{\boldsymbol{k}}\}$ utilizando la ecuación
\begin{equation*}
\underline{\underline{\boldsymbol{\varepsilon}}} =
\boldsymbol{T} \underline{\underline{\boldsymbol{\varepsilon}_{\text{cil}}}}
\boldsymbol{T}^T
\end{equation*}

In [5]:
eps = T*eps_cil*T.T

Extraemos las componentes de las deformaciones del tensor $\underline{\underline{\boldsymbol{\varepsilon}}}$:

In [6]:
ex  =   eps[0,0]
ey  =   eps[1,1]
ez  =   eps[2,2]
gxy = 2*eps[0,1]
gxz = 2*eps[0,2]
gyz = 2*eps[1,2]

Definimos las derivadas en coordenadas rectangulares:

In [7]:
dr_dx =    cos(t);   dr_dy = sin(t)
dt_dx = -sin(t)/r;   dt_dy = cos(t)/r

# se definen las primeras derivadas utilizando la regla de la cadena
d_dx = lambda f: diff(f,r)*dr_dx + diff(f,t)*dt_dx
d_dy = lambda f: diff(f,r)*dr_dy + diff(f,t)*dt_dy
d_dz = lambda f: diff(f,z)

# se definen las segundas derivadas
d2_dx2  = lambda f: d_dx(d_dx(f))
d2_dy2  = lambda f: d_dy(d_dy(f))
d2_dz2  = lambda f: d_dz(d_dz(f))
d2_dxdy = lambda f: d_dx(d_dy(f))
d2_dxdz = lambda f: d_dx(d_dz(f))
d2_dydz = lambda f: d_dy(d_dz(f))

Escribimos las ecuaciones de compatibilidad de Saint-Venant en coordenadas rectangulares:
\begin{align*}
\frac{\partial^2 \varepsilon_x}{\partial y^2}+\frac{\partial^2 \varepsilon_y}{\partial x^2} &= \frac{\partial^2 \gamma_{xy}}{\partial x \partial y} &  2\frac{\partial^2 \varepsilon_x}{\partial y \partial z} &= \frac{\partial}{\partial x}\left(-\frac{\partial \gamma_{yz}}{\partial x}+\frac{\partial \gamma_{xz}}{\partial y}+\frac{\partial \gamma_{xy}}{\partial z}\right) \\
\frac{\partial^2 \varepsilon_y}{\partial z^2}+\frac{\partial^2 \varepsilon_z}{\partial y^2} &= \frac{\partial^2 \gamma_{yz}}{\partial y \partial z} &  2\frac{\partial^2 \varepsilon_y}{\partial x \partial z} &= \frac{\partial}{\partial y}\left(\frac{\partial \gamma_{yz}}{\partial x}-\frac{\partial \gamma_{xz}}{\partial y}+\frac{\partial \gamma_{xy}}{\partial z}\right) \\
\frac{\partial^2 \varepsilon_z}{\partial x^2}+\frac{\partial^2 \varepsilon_x}{\partial z^2} &= \frac{\partial^2 \gamma_{xz}}{\partial x \partial z} &  2\frac{\partial^2 \varepsilon_z}{\partial x \partial y} &= \frac{\partial}{\partial z}\left(\frac{\partial \gamma_{yz}}{\partial x}+\frac{\partial \gamma_{xz}}{\partial y}-\frac{\partial \gamma_{xy}}{\partial z}\right);
\end{align*}
(tenga en cuenta que estas ecuaciones se escribieron de modo tal que sus lados derechos son iguales a cero)

In [8]:
e1a = d2_dy2(ex) + d2_dx2(ey) - d2_dxdy(gxy)
e2a = d2_dz2(ey) + d2_dy2(ez) - d2_dydz(gyz)
e3a = d2_dx2(ez) + d2_dz2(ex) - d2_dxdz(gxz)
e4a = 2*d2_dydz(ex) - d_dx(- d_dx(gyz) + d_dy(gxz) + d_dz(gxy))
e5a = 2*d2_dxdz(ey) - d_dy(+ d_dx(gyz) - d_dy(gxz) + d_dz(gxy))
e6a = 2*d2_dxdy(ez) - d_dz(+ d_dx(gyz) + d_dy(gxz) - d_dz(gxy))

Se hace $\sin(\theta)=0$ y $\cos(\theta)=1$, como en ocasiones anteriores:

In [9]:
e1a = simplify(e1a.subs({sin(t): 0, cos(t): 1}))
e2a = simplify(e2a.subs({sin(t): 0, cos(t): 1}))
e3a = simplify(e3a.subs({sin(t): 0, cos(t): 1}))
e4a = simplify(e4a.subs({sin(t): 0, cos(t): 1}))
e5a = simplify(e5a.subs({sin(t): 0, cos(t): 1}))
e6a = simplify(e6a.subs({sin(t): 0, cos(t): 1}))

Compararemos las ecuaciones obtenidas con las ecuaciones de Saint-Venant que aparecen en la literatura:
\begin{gather*}
        \frac{1}{r^2}\frac{\partial^2 \varepsilon_r}{\partial \theta^2} + \frac{1}{r}\frac{\partial}{\partial r} \left( r \frac{\partial \varepsilon_\theta}{\partial r} - \left( \varepsilon_r - \varepsilon_\theta \right) \right) = \frac{1}{r}\frac{\partial}{\partial \theta}\left(\frac{\partial \gamma_{r\theta}}{\partial r} + \frac{\gamma_{r \theta}}{r} \right) \\
        %
        %\allowdisplaybreaks
        \frac{1}{r^2}\frac{\partial^2 \varepsilon_z}{\partial \theta^2} + \frac{\partial^2 \varepsilon_\theta}{\partial z^2} + \frac{1}{r}\frac{\partial \varepsilon_z}{\partial r} = \frac{1}{r}\frac{\partial}{\partial z} \left( \frac{\partial \gamma_{\theta z}}{\partial \theta} + \gamma_{rz} \right) \\
        %
        %\allowdisplaybreaks
        \frac{\partial^2 \varepsilon_z}{\partial r^2} + \frac{\partial^2 \varepsilon_r}{\partial z^2} = \frac{\partial^2 \gamma_{rz}}{\partial r \partial z} \\
        %
        %\allowdisplaybreaks
        \frac{2}{r}\frac{\partial^2 \varepsilon_r}{\partial \theta \partial z} = 
        \frac{1}{r} \frac{\partial}{\partial r} \left( r\frac{\partial \gamma_{r\theta}}{\partial z} - \gamma_{\theta z} \right) + \frac{\partial}{\partial r} \left( \frac{1}{r}\frac{\partial \gamma_{rz}}{\partial \theta} - \frac{\partial\gamma_{\theta z}}{\partial r} \right) + \frac{
            1
        }{r}\frac{\partial \gamma_{r\theta}}{\partial z} + \frac{\gamma_{\theta z}}{r^2} \\
        %
        2\frac{\partial}{\partial z} \left( \frac{\partial \varepsilon_\theta}{\partial r} - \left( \frac{\varepsilon_r - \varepsilon_\theta}{r} \right) \right) = \frac{1}{r}\frac{\partial}{\partial \theta} \left( \frac{\partial \gamma_{\theta z}}{\partial r} - \frac{1}{r}\frac{\partial \gamma_{rz}}{\partial \theta} + \frac{\partial \gamma_{r \theta}}{\partial z} \right) + \frac{1}{r^2}\frac{\partial \gamma_{\theta z}}{\partial \theta} \\
        %
        \frac{2}{r}\frac{\partial}{\partial \theta} \left( \frac{\partial \varepsilon_z}{\partial r} - \frac{\varepsilon_z}{r} \right) = \frac{\partial}{\partial z} \left( \frac{\partial \gamma_{\theta z}}{\partial r} + \frac{1}{r}\frac{\partial \gamma_{rz}}{\partial \theta} - \frac{\partial \gamma_{r \theta}}{\partial z} - \frac{\gamma_{\theta z}}{r} \right).
\end{gather*}

In [10]:
e1b = (1/r**2)*diff(er,t,2) + (1/r)*diff(r*diff(et,r) - (er-et),r) - (1/r)*diff(diff(grt,r)  + grt/r,t)
e2b = (1/r**2)*diff(ez,t,2) + diff(et,z,2) + (1/r)*diff(ez,r) - (1/r)*diff(diff(gtz,t) + grz,z)
e3b = diff(ez,r,2) + diff(er,z,2) - diff(grz,r,1,z,1)
e4b = (2/r)*diff(er,t,1,z,1) - (1/r)*diff(r*diff(grt,z) - gtz,r) - diff((1/r)*diff(grz,t) - diff(gtz,r),r) - (1/r)*diff(grt,z) - gtz/r**2
e5b = 2*diff(diff(et,r) - (er-et)/r,z) - (1/r)*diff(diff(gtz,r) - (1/r)*diff(grz,t) + diff(grt,z),t) - (1/r**2)*diff(gtz,t)
e6b = (2/r)*diff(diff(ez,r) - ez/r,t) - diff(diff(gtz,r) + (1/r)*diff(grz,t) - diff(grt,z) - gtz/r,z)

Observamos que se obtuvo el vector de ceros, lo que confirma que acabamos de calcular las ecuaciones de Saint-Venant en coordenadas cilindricas:

In [11]:
expand(Matrix([ e1a - e1b,
                e2a - e2b,
                e3a - e3b,
                e4a - e4b,
                e5a - e5b,
                e6a - e6b ]))

Matrix([
[0],
[0],
[0],
[0],
[0],
[0]])